In [1]:
import os
import pandas as pd
import dataframe_image as dfi
from PIL import Image, ImageDraw, ImageFont
from gcd_table import build_gcd_table_filled

In [2]:
# example from:
#  https://win-vector.com/2024/10/06/dudeneys-remainder-problem/
soln = build_gcd_table_filled(
    508811 - 480608, 
    723217 - 480608,
    verbose=True)

'build row 0: start (a >= b)'

,a,b,a%b,a//b,"GCD(a, b)"
0,242609,28203,16985,8,
1,,,,,
2,,,,,
3,,,,,
4,,,,,
5,,,,,
6,,,,,
7,,,,,


'build row 1: a[1]=b[0], b[1]=(a%b)[0]'

,a,b,a%b,a//b,"GCD(a, b)"
0,242609,28203,16985,8,
1,28203,16985,11218,1,
2,,,,,
3,,,,,
4,,,,,
5,,,,,
6,,,,,
7,,,,,


'build row 2: a[2]=b[1], b[2]=(a%b)[1]'

...

,a,b,a%b,a//b,"GCD(a, b)",u,v
0,242609,28203,16985,8,79,,
1,28203,16985,11218,1,79,,
2,16985,11218,5767,1,79,-35,53
3,11218,5767,5451,1,79,18,-35
4,5767,5451,316,1,79,-17,18
5,5451,316,79,17,79,1,-17
6,316,79,0,4,79,0,1
7,79,0,N/A,N/A,79,1,0


'back fill row 1: u[1]=v[2], v[1] = u[2] - (a//b)[1] * v[2]'

,a,b,a%b,a//b,"GCD(a, b)",u,v
0,242609,28203,16985,8,79,,
1,28203,16985,11218,1,79,53,-88
2,16985,11218,5767,1,79,-35,53
3,11218,5767,5451,1,79,18,-35
4,5767,5451,316,1,79,-17,18
5,5451,316,79,17,79,1,-17
6,316,79,0,4,79,0,1
7,79,0,N/A,N/A,79,1,0


'back fill row 0: u[0]=v[1], v[0] = u[1] - (a//b)[0] * v[1]'

,a,b,a%b,a//b,"GCD(a, b)",u,v
0,242609,28203,16985,8,79,-88,757
1,28203,16985,11218,1,79,53,-88
2,16985,11218,5767,1,79,-35,53
3,11218,5767,5451,1,79,18,-35
4,5767,5451,316,1,79,-17,18
5,5451,316,79,17,79,1,-17
6,316,79,0,4,79,0,1
7,79,0,N/A,N/A,79,1,0


In [3]:

tables = list(soln.attrs['captured_tables'])  # new list, so we can append without side effects

In [4]:
soln['u * a + v * b'] = soln['u'] * soln['a'] + soln['v'] * soln['b']
soln.attrs['note'] = "Show Bezout's identity."

def highlight_col(x):
    return ['background-color: yellow; font-weight: bold' if col_name == 'u * a + v * b' else '' for col_name in x.index]


tables.append(soln.style.apply(highlight_col, axis=1))

In [5]:
for i, t in enumerate(tables):
    file_name = f'dataframe_{i:04d}.png'
    # convert to PNG
    dfi.export(t, file_name, dpi=600)


In [6]:
# add annotation notes
font = ImageFont.load_default(size=50)
text_position = (50, 10)  # x, y coordinates for the text
text_color = (0, 0, 0)
for i, t in enumerate(tables):
    file_name = f'dataframe_{i:04d}.png'
    file_name_out = f'dataframe_annotated_{i:04d}.png'
    note = t.data.attrs['note']
    # add in text or copy image  
    image = Image.open(file_name)
    draw = ImageDraw.Draw(image)
    if note is not None:
        draw.text(text_position, note, fill=text_color, font=font)
    image.save(file_name_out)

Then convert at the command line with:

```
ffmpeg -framerate 0.5 -i dataframe_annotated_%04d.png -c:v libx264 -r 30 -pix_fmt yuv420p RemainderProblem.mp4
```


In [7]:
!ffmpeg -framerate 0.5 -i dataframe_annotated_%04d.png -c:v libx264 -r 30 -pix_fmt yuv420p RemainderProblem.mp4

ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/5.1.2_6 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --

In [8]:
# clean up
for i, t in enumerate(tables):
    file_name = f'dataframe_{i:04d}.png'
    file_name_out = f'dataframe_annotated_{i:04d}.png'
    os.remove(file_name)
    os.remove(file_name_out)